In [1]:
!git clone https://github.com/NLP-Final-Projects/legal_case_rag

Cloning into 'legal_case_rag'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 55 (delta 11), reused 48 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (55/55), 1.66 MiB | 3.92 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Error downloading object: dataset/case_df_part1.csv (ec0cab2): Smudge error: Error downloading dataset/case_df_part1.csv (ec0cab28bddcd34663d7ad5078eaf79bb8ec6af286a09a9d0dfb6423d3ea0209): batch response: This repository is over its data quota. Account responsible for LFS bandwidth should purchase more data packs to restore access.

Errors logged to /content/legal_case_rag/.git/lfs/logs/20240924T045134.860077165.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: dataset/case_df_part1.csv: smudge filter lfs failed
You can inspect what was checked out with 'git status'
and retry with 'git restore --

In [1]:
!chmod -R 777 /content/legal_case_rag/

In [2]:
%cd legal_case_rag/


/content/legal_case_rag


In [3]:
!pip install -r Requirements.txt

In [4]:
!pip install langchain_openai sentence-transformers chromadb hazm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Download Chromadb and preprocessed legal cases csv files

In [5]:
%cd /content/legal_case_rag
!wget -O processed_cases.csv "https://drive.usercontent.google.com/download?id=1jMuQtywo0mbj7ZHCCsyE8xurbSyVVCst&export=download&confirm=t&uuid=2f681c98-86f8-4159-9e03-673cdcbc7cb51"
!wget -O chromadb_collection.rar "https://drive.usercontent.google.com/download?id=1-3N-QjK6QMogy2mcNEKjTg_9yIrSjh4A&export=download&confirm=t&uuid=ceed27ad-0f81-4c16-b70f-68cebcc55279"
!unrar x chromadb_collection.rar
!mv content/chromadb_collections chromadb_collections
!rm -r content

/content/legal_case_rag
--2024-09-24 04:53:06--  https://drive.usercontent.google.com/download?id=1jMuQtywo0mbj7ZHCCsyE8xurbSyVVCst&export=download&confirm=t&uuid=2f681c98-86f8-4159-9e03-673cdcbc7cb51
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.101.132, 2607:f8b0:4023:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.101.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 453684627 (433M) [application/octet-stream]
Saving to: ‘processed_cases.csv’

processed_cases.csv 100%[===================>] 432.67M  91.9MB/s    in 6.3s    

2024-09-24 04:53:14 (68.9 MB/s) - ‘processed_cases.csv’ saved [453684627/453684627]

--2024-09-24 04:53:14--  https://drive.usercontent.google.com/download?id=1-3N-QjK6QMogy2mcNEKjTg_9yIrSjh4A&export=download&confirm=t&uuid=ceed27ad-0f81-4c16-b70f-68cebcc55279
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.101.132, 2607:f8b0:40

# Setup Files & Classes

## RAG Class

In [6]:
%%writefile model/rag/rag_handler.py

from typing import List
import chromadb
from transformers import AutoTokenizer, AutoModel
from chromadb.config import Settings
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from hazm import *


class RAG:
    def __init__(self,
                 model_name: str = "HooshvareLab/bert-base-parsbert-uncased",
                 collection_name: str = "legal_cases",
                 persist_directory: str = "chromadb_collections/",
                 top_k: int = 2
                 ) -> None:

        self.cases_df = pd.read_csv('processed_cases.csv')

        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.normalizer = Normalizer()
        self.top_k = top_k

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)

        self.client = chromadb.PersistentClient(path=persist_directory)

        self.collection = self.client.get_collection(name=collection_name)

    def query_pre_process(self, query: str) -> str:
        return self.normalizer.normalize(query)

    def embed_single_text(self, text: str) -> np.ndarray:
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        inputs = {key: value.to(self.device) for key, value in inputs.items()}
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()


    def extract_case_title_from_df(self, case_id: str) -> str:

        case_id_int = int(case_id.split("_")[1])

        try:
            case_title = self.cases_df.loc[case_id_int, 'title']
            return case_title
        except KeyError:
            return "Case ID not found in DataFrame."

    def extract_case_text_from_df(self, case_id: str) -> str:
        case_id_int = int(case_id.split("_")[1])

        try:
            case_text = self.cases_df.loc[case_id_int, 'text']
            return case_text
        except KeyError:
            return "Case ID not found in DataFrame."

    def retrieve_relevant_cases(self, query_text: str) -> List[str]:
        normalized_query_text = self.query_pre_process(query_text)

        query_embedding = self.embed_single_text(normalized_query_text)
        query_embedding_list = query_embedding.tolist()

        results = self.collection.query(
            query_embeddings=[query_embedding_list],
            n_results=self.top_k
        )

        retrieved_cases = []
        for i in range(len(results['metadatas'][0])):
            case_id = results['ids'][0][i]
            case_text = self.extract_case_text_from_df(case_id)
            case_title = self.extract_case_title_from_df(case_id)
            retrieved_cases.append({
                "text": case_text,
                "title": case_title
            })

        return retrieved_cases

    def get_information(self, query: str) -> List[str]:
        return self.retrieve_relevant_cases(query)

Overwriting model/rag/rag_handler.py


In [7]:
# !python model/rag/rag_handler.py

## Prompt Class

In [8]:
%%writefile model/propmt/prompt_handler.py
from typing import List

class Prompt:

    def get_prompt(self, message:str, info_list: List) -> str:
        prompt = f"As a user, I want to ask you the following legal question:\n{message}\n\n"

        if info_list:
            prompt += "Here are some relevant legal cases and information you should consider:\n"
            for i, info in enumerate(info_list):
                prompt += f"case {i+1}:\n{info['title']}\n{info['text']}\n"

        prompt += "\nBased on the provided information, please respond in Persian(Farsi) with a concise legal analysis.\
                    Ensure that your response is as summarized and clear as possible. (one paragraph)"

        return prompt

Overwriting model/propmt/prompt_handler.py


## LLM_API_Call Class

In [9]:
%%writefile model/llm/llm.py

from langchain_openai import OpenAI
import openai
import sys
import os
import requests


sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '../../')))

from config import *


class LLM_API_Call:

    def __init__(self, type) -> None:
        if type == "openai":
            self.llm = OpenAI_API_Call(api_key = LLM_CONFIG[""],
                                       model = LLM_CONFIG["model"])
        elif type == "gilas":
            self.llm = Gilas_API_Call(api_key = GILAS_CONFIG["api_key"],
                                       model = GILAS_CONFIG["model"],
                                       base_url=GILAS_CONFIG["base_url"])
        else:
            self.llm = OpenAI(
                            **LLM_CONFIG
            )

    def get_LLM_response(self, prompt: str) -> str:
        return self.llm.invoke(prompt)
    

class AvalAI:
    
    def __init__(self, model: str, base_url: str, api_key: str) -> None:
        self.model_name = model
        self.base_url = base_url
        self.api_key = api_key
        self.conversation = []
        self._model = ChatOpenAI(
                model=model,
                # temperature=0,
                # max_tokens=None,
                # timeout=None,
                # max_retries=2,
                api_key=api_key,  # if you prefer to pass api key in directly instaed of using env vars
                base_url=base_url,
            )
        
    def get_LLM_response(self, prompt: str, status: str) -> str:
        
        # Extend the conversation history with the new prompt
        self.conversation.append({"role": "user", "content": prompt})
        
        # Combine all conversation history into a single input
        full_conversation = [{"role": "system", "content": status}] + self.conversation
        
        # Get the model's response based on the full conversation history
        response = self._model.invoke(full_conversation)
        
        # Since the response is likely an object, access its content directly
        assistant_response = response.content  # Accessing the content directly
        
        # Add the assistant's response to the conversation history
        self.conversation.append({"role": "assistant", "content": assistant_response})
        return assistant_response



class OpenAI_API_Call:

    def __init__(self, api_key, model="gpt-4"):
        self.api_key = api_key
        openai.api_key = api_key
        self.model = model
        self.conversation = []

    def add_message(self, role, content):
        self.conversation.append({"role": role, "content": content})

    def get_response(self):
        response = openai.ChatCompletion.create(
            model=self.model,
            messages=self.conversation
        )
        return response['choices'][0]['message']['content']

    def invoke(self, user_input):
        self.add_message("user", user_input)

        response = self.get_response()

        self.add_message("assistant", response)

        return response


class Gilas_API_Call:
    def __init__(self, api_key, base_url, model="gpt-4o-mini"):
        self.api_key = api_key
        self.base_url = base_url
        self.model = model
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        self.conversation = []

    def add_message(self, role, content):
        self.conversation.append({"role": role, "content": content})

    def get_response(self):
        data = {
            "model": self.model,
            "messages": self.conversation
        }

        response = requests.post(
            url=f"{self.base_url}/chat/completions",
            headers=self.headers,
            json=data
        )

        if response.status_code == 200:
            try:
                return response.json()['choices'][0]['message']['content']
            except (KeyError, IndexError, ValueError) as e:
                raise Exception(f"Unexpected API response format: {e}")
        else:
            raise Exception(f"Gilas API call failed: {response.status_code} - {response.text}")

    def invoke(self, user_input, max_retries=3, initial_wait=30):
        self.add_message("user", user_input)

        retries = 0
        wait_time = initial_wait
        while retries < max_retries:
            try:
                response = self.get_response()
                self.add_message("assistant", response)
                return response
            except (JSONDecodeError, Exception) as e:
                print(f"Error encountered: {e}. Retrying in {wait_time} seconds...")
                retries += 1
                time.sleep(wait_time)
                wait_time += 30
        raise Exception(f"Failed to get a valid response after {max_retries} retries.")


Overwriting model/llm/llm.py


## Chat Class

In [10]:
%%writefile model/chat.py

from model.propmt.prompt_handler import *
from model.llm.llm import *
from model.rag.rag_handler import *
from config import *

class Chat:
    def __init__(self, chat_id, rag_handler) -> None:
        self.chat_id = chat_id
        self.message_history = []
        self.response_history = []
        self.prompt_handler = Prompt()
        self.llm = LLM_API_Call("gilas")
        self.rag_handler = rag_handler

    def response(self, message: str) -> str:
        self.message_history.append(message)

        info_list = self.rag_handler.get_information(message)
        prompt = self.prompt_handler.get_prompt(message, info_list)
        response = self.llm.get_LLM_response(prompt=prompt)

        self.response_history.append(response)
        return response


Overwriting model/chat.py


## Config File

In [11]:
%%writefile config.py

gpt_3_5 = "gpt-3.5-turbo-instruct"
gpt_mini = "gpt-4o-mini"

aval_ai = {
    "model": gpt_3_5,
    "base_url": "https://api.avalai.ir/v1",

}

GILAS_CONFIG = {
    "model": "gpt-4o-mini" ,
    "base_url": 'https://api.gilas.io/v1',
}

OPENAI_CONFIG = {
    "model": gpt_mini,
}


LLM_CONFIG = aval_ai


Overwriting config.py


## Controller Class

In [12]:
%%writefile model/controller.py
from model.chat import *

sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '../../')))

class Controller:
    def __init__(self) -> None:
        self.chat_dic = {}
        self.rag_handler = RAG()

    def handle_message(self,
                       chat_id: int,
                       message: str) -> str:
        if chat_id not in self.chat_dic:
             self.chat_dic[chat_id] = Chat(chat_id=chat_id, rag_handler=self.rag_handler)
        chat = self.chat_dic[chat_id]
        return chat.response(message)


Overwriting model/controller.py


In [13]:
!pwd

/content/legal_case_rag


# Testing Phase

## Testing the chatbot (terminal mode)

In [14]:
import pandas as pd
from model.controller import Controller
import time


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


bot = Controller()
while True:
    message = input("Enter your law question (or type 'exit' to quit): ")

    if message.lower() == "exit":
        print("Exiting the chat.")
        break

    res = bot.handle_message(2311, message)

    print(res)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

Enter your law question (or type 'exit' to quit): در صورتی که زوجه محجور اقدام به مطالبه‌ی مهریه کند، چه ملاحظات قانونی و رویه‌های قضائی برای پیگیری چنین پرونده‌ای وجود دارد؟
اگر زوجه محجور اقدام به مطالبه‌ی مهریه کند، باید به ملاحظات قانونی مربوط به وضعیت محجوریت او و حق حاکمیت زن بر مهریه‌اش توجه شود. در حقوق ایران، به استناد ماده ۱۰۸۲ قانون مدنی، مهریه حق مسلم زوجه است و در صورتی که زوجه محجور باشد، اقدام به دریافت مهریه تحت نظارت و تأیید ولی یا قیم او امکان‌پذیر است. در رویه‌ قضائی نیز، مانند موارد ذکر شده در رأی‌های دادگاه، تمرکز بر حقایق و شرایط خاص هر پرونده مؤثر است؛ از جمله بررسی عسر و حرج زوجه و وضعیت خانوادگی او. لذا، در مواجهه با چنین پرونده‌ای، نیاز به احراز قانونی بودن درخواست و امکان عملی آن توسط ولی یا قیم زوجه وجود دارد. همچنین، دادگاه‌ها در ارزیابی شرایط و تمایل زوجه به زندگی مشترک و تأثیرات آن بر حق مهریه منفک می‌شوند.
Enter your law question (or type 'exit' to quit): exit
Exiting the chat.


## Testing the chatbot with test data (.csv files) (terminal mode)

In [ ]:
import pandas as pd
from model.controller import Controller
import time

bot = Controller()

eval_df = pd.read_csv('evaluation/test/easy.csv')

eval_df['rag_answer'] = ''

output_file = 'easy_ans.csv'

for index, row in eval_df.iterrows():
    chat_id = index
    message = row['question']

    chatbot_answer = bot.handle_message(chat_id=chat_id, message=message)

    print("Question: ", message)
    print("RAG Chatbot answer: ", chatbot_answer)

    eval_df.at[index, 'rag_answer'] = chatbot_answer

    eval_df.to_csv(output_file, index=False, encoding='utf-8')

    print(f'Processed row {index + 1}/{len(eval_df)}/n')

    time.sleep(60)

print(f'Processing complete. Results saved to {output_file}')


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Question:  در صورتی که زوجه محجور اقدام به مطالبه‌ی مهریه کند، چه ملاحظات قانونی و رویه‌های قضائی برای پیگیری چنین پرونده‌ای وجود دارد؟
RAG Chatbot answer:  در صورتی که زوجه محجور اقدام به مطالبه‌ی مهریه کند، باید به دو نکته‌ی اساسی توجه کرد: اولاً، طبق ماده ۱۰۸۲ قانون مدنی، مهریه حق مسلم زوجه است و نیازی به موافقت زوج ندارد، بنابراین حتی در شرایط محجوریت زوجه، او می‌تواند درخواست مطالبه‌ی مهریه کند. ثانیاً، دادگاه باید به وضعیت فردی زوجه و دلایل محجوریت او توجه کند و در تصمیم‌گیری، آثار و تبعات این محجوریت را در نظر بگیرد. همچنین، رویه‌ی قضائی نشان می‌دهد که در دعاوی مرتبط با مهریه، عدم امکان تمکین و عسر و حرج زن در زندگی مشترک از عوامل تأثیرگذار بر رأی دادگاه است و اعتراضات به موقع و صحیح طرفین می‌تواند بر روند دادرسی تأثیرگذار باشد. لذا زوجه با تکیه بر حقوق قانونی خود و استناد به مستندات مربوطه، می‌تواند خواسته‌ی خود را پیگیری کند.
Processed row 1/22
Question:  اگر قرارداد بین طرفین اقاله شده و طرف مقابل به تعهدات خود عمل نکرده و وجه التزام را پرداخت نمی‌کند، چگونه می‌توانم مطالبه و

Exception: Unexpected API response format: Expecting value: line 1 column 1 (char 0)

## Handle API Rate Limit (429 status code)

In [ ]:
import pandas as pd
from model.controller import Controller
import time
import json
from json.decoder import JSONDecodeError

bot = Controller()

eval_df = pd.read_csv('evaluation/test/easy.csv')

eval_df['rag_answer'] = ''

output_file = 'easy_ans.csv'

for index, row in eval_df.iterrows():
    chat_id = index
    message = row['question']

    try:
        chatbot_answer = bot.handle_message(chat_id=chat_id, message=message)

        print("Question: ", message)
        print("RAG Chatbot answer: ", chatbot_answer)

        eval_df.at[index, 'rag_answer'] = chatbot_answer

        eval_df.to_csv(output_file, index=False, encoding='utf-8')

        print(f'Processed row {index + 1}/{len(eval_df)}')

    except Exception as e:
        print(f"Failed to get an answer for question at index {index}. Error: {e}")

    time.sleep(60)

print(f'Processing complete. Results saved to {output_file}')

In [15]:
import pandas as pd
from model.controller import Controller
import time
import os
import json
from json.decoder import JSONDecodeError

bot = Controller()

eval_df = pd.read_csv('evaluation/test/easy.csv')

output_file = 'easy_ans.csv'

if os.path.exists(output_file):
    existing_df = pd.read_csv(output_file)
    eval_df['rag_answer'] = ''
    # Combine answers from existing file with the new dataframe
    if 'rag_answer' in existing_df.columns:
        eval_df['rag_answer'] = existing_df['rag_answer'].combine_first(eval_df['rag_answer'])
    else:
        eval_df['rag_answer'] = ''
else:
    eval_df['rag_answer'] = ''

for index, row in eval_df.iterrows():
    chat_id = index
    message = row['question']

    # Check if an answer already exists
    if pd.notna(row['rag_answer']) and row['rag_answer'].strip() != '':
        print(f"Answer already exists for question at row {index + 1}. Skipping chatbot request.")
        continue  # Skip to the next question if an answer already exists

    try:
        # Get the chatbot's answer with retry logic handled internally
        chatbot_answer = bot.handle_message(chat_id=chat_id, message=message)

        print("Question: ", message)
        print("RAG Chatbot answer: ", chatbot_answer)

        eval_df.at[index, 'rag_answer'] = chatbot_answer

        eval_df.to_csv(output_file, index=False, encoding='utf-8')

        print(f'Processed row {index + 1}/{len(eval_df)}')

    except Exception as e:
        print(f"Failed to get an answer for question at row {index + 1} after retries. Skipping this question.")

    time.sleep(60)

print(f'Processing complete. Results saved to {output_file}')


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Question:  در صورتی که زوجه محجور اقدام به مطالبه‌ی مهریه کند، چه ملاحظات قانونی و رویه‌های قضائی برای پیگیری چنین پرونده‌ای وجود دارد؟
RAG Chatbot answer:  در صورتی که زوجه محجور (یعنی زوجه‌ای که به دلیل بیماری یا وضعیت‌های قانونی دیگر نمی‌تواند به طور مستقل اقدام کند) اقدام به مطالبه‌ی مهریه کند، نیاز است که به ملاحظات قانونی و رویه‌های قضائی توجه شود. بر اساس ماده ۱۰۸۲ قانون مدنی، مهریه حق مسلم زوجه بوده و در صورت عدم پرداخت آن، زوجه می‌تواند از طریق دادگاه برای دریافت مهریه اقدام نماید. همچنین، احراز عسر و حرج زوجه پیش از زندگی مشترک می‌تواند در تعیین حقانیت مطالبات وی موثر باشد. قضات می‌توانند با توجه به وضعیت محجوریت زوجه و توان او در همکاری در فرآیند قضائی، به پرونده رسیدگی کنند. لذا تاکید بر این است که تمامی مستندات و مدارک قانونی باید به دقت بررسی و مورد ارزیابی قرار گیرد تا نتیجه عادلانه‌ای حاصل شود.
Processed row 1/22


KeyboardInterrupt: 

In [ ]:
# import pandas as pd
# cases_df = pd.read_csv('processed_cases.csv')
# cases_df.head()

,title,number,date,vote_type,text,rm_names,rm_d_n
0,مطالبه وجه التزام پس از اقاله قرارداد,9409970908300837,1394/12/17,حقوقی,رأی خلاصه جریان پرونده. شماره پرونده: ۹۱۰۹۹۸۰۳...,رأی خلاصه جریان پرونده. شماره پرونده: ۹۱۰۹۹۸۰۳...,رأی خلاصه جریان پرونده.. خلاصه جریان پرونده. ب...
1,مطالبه ی مهریه از جانب زوجه محجور,9409970908300834,1394/12/15,حقوقی,رأی خلاصه جریان پرونده. در تاریخ ۱۳۸۳ / ۰۶ / ۲...,رأی خلاصه جریان پرونده. در تاریخ ۱۳۸۳ / ۰۶ / ۲...,رأی خلاصه جریان پرونده. دادخواستی به طرفیت به ...
2,طرح دعوی به خواسته ثبوت عدم تمکین زوجه,8909982330300949,1395/02/01,حقوقی,رأی خلاصه جریان پرونده.. \nدر تاریخ ۸۸ / ۷ / ۲...,رأی خلاصه جریان پرونده.. \nدر تاریخ ۸۸ / ۷ / ۲...,رأی خلاصه جریان پرونده.. به عقد دائمی همسرش در...
3,مهلت رجوع زوجه به مابذل در طلاق خلع,9409970908300823,1394/12/09,حقوقی,رأی خلاصه جریان پرونده. پرونده‌های محاکماتی کل...,رأی خلاصه جریان پرونده. پرونده‌های محاکماتی کل...,رأی خلاصه جریان پرونده. پرونده‌های محاکماتی شع...
4,تغییر شغل و تغییر در عین مستاجره,9409970908300843,1394/12/19,حقوقی,رأی خلاصه جریان پرونده. درتاریخ ۱۳۸۷ / ۰۷ / ۰۴...,رأی خلاصه جریان پرونده. درتاریخ ۱۳۸۷ / ۰۷ / ۰۴...,رأی خلاصه جریان پرونده. وسید و و وب. همگی با و...


In [ ]:
# !rm -r chromadb_collections/

In [ ]:
# !python model/rag/rag_handler.py

config.json: 100% 434/434 [00:00<00:00, 2.13MB/s]
vocab.txt: 100% 1.22M/1.22M [00:00<00:00, 6.03MB/s]
pytorch_model.bin: 100% 654M/654M [00:08<00:00, 81.5MB/s]
[{'text': 'رأی دادگاه بدوی. در رابطه با اتهام آقای م. کارمند بانک مبنی بر اختلاس به میزان سیصد میلیون ریال معادل سی میلیون تومان از وجوه در اختیار ایشان در بانک بدین نحو که مشارالیه به جهت بدهکاری به احدی از مشتریان بانکی از حساب واسطه بانک مبلغ فوق الاشاره را برداشت و به حساب مشتری جهت اداء دین شخصی واریز می\u200cنماید و علی\u200cرغم تذکرات مراجع نظارتی نسبت به جبران خسارات وارده اقدام نمی\u200cنماید تا این\u200cکه موضوع به مراجع قضایی اعلام و قبل از صدور کیفرخواست مبادرت به پرداخت خسارات می\u200cنماید. دادگاه ضمن احراز بزه انتسابی اما با توجه به رأی هیئت تجدیدنظر رسیدگی به تخلفات و استحقاق تخفیف به استناد تبصره ۳ از ماده ۵ قانون تشدید مجازات مرتکبین ارتشاء و اختلاس و کلاهبرداری و ماده ۲۲ قانون مجازات اسلامی از جهت ابراز ندامت، سعی در جبران خسارات وی را به یک سال حبس تعزیری، شش\u200cصد میلیون ریال جزای نقدی و یک سال انفصال از خ

In [ ]:
# rag = RAG(top_k=5)
# query = "آیا حکم استرداد ملکی که متهم به فروش مال غیر بوده است، در صورتی که دادگاه حکم به برائت او از اتهام فروش مال غیر داده باشد، قانونی است؟"
# results = rag.ge(query)

# # Display the results
# print(results)